In [1]:
import pulp as plp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, LpInteger, LpContinuous, LpBinary, LpMinimize
from domain.entities import DayOfTheWeek, Employee, Patient, Combination, PersonList
import itertools

### Employee Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Employee 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 07:30 - 8:00  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:00 - 8:30  | G1 | G2 | -- | G1 | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:00 - 10:30 | G2 | G3 | -- | G3 | -- | -- | -- |
| 10:30 - 11:00 | G2 | G3 | -- | G3 | -- | -- | -- |
| 11:00 - 11:30 | G1 | G3 | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | G1 | G3 | -- | G3 | -- | -- | -- |

| Employee 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G1 | G2 | -- | -- | -- |
| 07:30 - 8:00  | -- | G1 | G3 | G2 | -- | -- | -- |
| 08:00 - 8:30  | -- | G1 | G3 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | G3 | G1 | G2 | -- | -- | -- | -- |
| 10:30 - 11:00 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:00 - 11:30 | G3 | -- | G2 | G1 | -- | -- | -- |
| 11:30 - 12:00 | G3 | -- | G2 | G1 | -- | -- | -- |

| Employee 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | G2 | -- | G2 | -- | -- |
| 07:30 - 8:00  | -- | G3 | G2 | -- | G2 | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | G2 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | G1 | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | G1 | -- | G1 | -- | -- |
| 10:00 - 10:30 | -- | G1 | G1 | -- | G1 | -- | -- |
| 10:30 - 11:00 | -- | G1 | G1 | -- | G1 | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Occupation | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | G1----- | G2----G3 | ---G1-G2 | G1-G2--- | G3----G2 | -- | -- |
| 07:30 - 8:00  | G1----- | G2-G1-G3 | ---G3-G2 | G1-G2--- | G3----G2 | -- | -- |
| 08:00 - 8:30  | G1----- | G2-G1--- | ---G3-G2 | G1------ | G3----G2 | -- | -- |
| 08:30 - 9:00  | ------- | -------- | -------- | -------- | G3------ | -- | -- |
| 09:00 - 9:30  | ------- | ------G1 | -------- | -------- | G3----G3 | -- | -- |
| 09:30 - 10:00 | G2-G3-- | G3-G1--- | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:00 - 10:30 | G2-G3-- | G3-G1-G1 | ---G2-G1 | G3------ | ------G1 | -- | -- |
| 10:30 - 11:00 | G2-G3-- | G3----G1 | ---G2-G1 | G3-G1--- | ------G1 | -- | -- |
| 11:00 - 11:30 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |
| 11:30 - 12:00 | G1-G3-- | G3------ | ---G2--- | G3-G1--- | -------- | -- | -- |





### Patient Schedules
We limited the schedules from 7:00 to 12:00 for clarity
| Patient 1 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | G2 | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | G2 | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | G1 | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 2 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | G3 | -- | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | G3 | -- | -- | G3 | -- | -- |
| 08:00 - 8:30  | -- | -- | -- | -- | G3 | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | G3 | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | G3 | -- | -- |
| 09:30 - 10:00 | -- | -- | -- | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | -- | -- | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | -- | -- | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | -- | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |

| Patient 3 | Monday | Tuesday | Wednesday | Thursday | Friday | Saturday | Sunday |
| --- | --- | --- | --- | --- | --- | --- | --- |
| 07:00 - 7:30  | -- | -- | G2 | -- | -- | -- | -- |
| 07:30 - 8:00  | -- | -- | G2 | -- | -- | -- | -- |
| 08:00 - 8:30  | -- | -- | G2 | -- | -- | -- | -- |
| 08:30 - 9:00  | -- | -- | -- | -- | -- | -- | -- |
| 09:00 - 9:30  | -- | -- | -- | -- | -- | -- | -- |
| 09:30 - 10:00 | -- | -- | G2 | -- | -- | -- | -- |
| 10:00 - 10:30 | -- | -- | G2 | G3 | -- | -- | -- |
| 10:30 - 11:00 | -- | -- | G2 | G3 | -- | -- | -- |
| 11:00 - 11:30 | -- | -- | -- | G3 | -- | -- | -- |
| 11:30 - 12:00 | -- | -- | -- | -- | -- | -- | -- |


In [3]:
# define dummy employees
employee1 = Employee(1)
employee2 = Employee(2)
employee3 = Employee(3)
employee4 = Employee(4)
employee5 = Employee(5)
employee6 = Employee(6)
employee7 = Employee(7)
employee8 = Employee(8)
employee9 = Employee(9)
employee10 = Employee(10)

employee1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 7,8.5),
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.MONDAY, 12.5,14),
        (1, DayOfTheWeek.TUESDAY, 7,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,12),
        (1, DayOfTheWeek.THURSDAY,7,8.5),
        (1, DayOfTheWeek.THURSDAY,9.5,12),
        (1, DayOfTheWeek.FRIDAY, 7,9.5),
    ])

employee2.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,12),
        (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
        (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
        (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
        (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
        (1, DayOfTheWeek.WEDNESDAY,9.5,12),
        (1, DayOfTheWeek.THURSDAY, 7,8),
        (1, DayOfTheWeek.THURSDAY, 10.5,12)
    ])

employee3.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee4.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employee5.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
    (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
    (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,12),
    (1, DayOfTheWeek.THURSDAY, 7,8),
    (1, DayOfTheWeek.THURSDAY, 10.5,12)
])

employee6.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee7.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employee8.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.TUESDAY, 7.5,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,10.5),
    (1, DayOfTheWeek.WEDNESDAY, 7,7.5),
    (1, DayOfTheWeek.WEDNESDAY,7.5,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,12),
    (1, DayOfTheWeek.THURSDAY, 7,8),
    (1, DayOfTheWeek.THURSDAY, 10.5,12)
])

employee9.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.TUESDAY,9,9.5),
    (1, DayOfTheWeek.TUESDAY,10,11),
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.FRIDAY,7,8.5),
    (1, DayOfTheWeek.FRIDAY,9,9.5),
    (1, DayOfTheWeek.FRIDAY,9.5,11),
])

employee10.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 7,8.5),
    (1, DayOfTheWeek.MONDAY, 9.5,12),
    (1, DayOfTheWeek.MONDAY, 12.5,14),
    (1, DayOfTheWeek.TUESDAY, 7,8.5),
    (1, DayOfTheWeek.TUESDAY, 9.5,12),
    (1, DayOfTheWeek.THURSDAY,7,8.5),
    (1, DayOfTheWeek.THURSDAY,9.5,12),
    (1, DayOfTheWeek.FRIDAY, 7,9.5),
])

employees = [employee1, employee2, employee3, employee4, employee5, employee6, employee7, employee8, employee9, employee10]


KeyError: '1DayOfTheWeek.MONDAY07.00'

In [122]:
# define dummy patients
patient1 = Patient(1)
patient2 = Patient(2)
patient3 = Patient(3)
patient4 = Patient(4)
patient5 = Patient(5)
patient6 = Patient(6)
patient7 = Patient(7)
patient8 = Patient(8)
patient9 = Patient(9)
patient10 = Patient(10)

patient1.schedule.add_items([
        (1, DayOfTheWeek.MONDAY, 9.5,10.5),
        (1, DayOfTheWeek.MONDAY, 11,11.5),
        (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient2.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient3.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient4.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient5.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient6.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient7.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
    ])

patient8.schedule.add_items([
    (1, DayOfTheWeek.TUESDAY,7,8),
    (1, DayOfTheWeek.FRIDAY,7.5,9.5),
    ])

patient9.schedule.add_items([
    (1, DayOfTheWeek.WEDNESDAY,7,8.5),
    (1, DayOfTheWeek.WEDNESDAY,9.5,11),
    (1, DayOfTheWeek.THURSDAY,10,11.5),
])

patient10.schedule.add_items([
    (1, DayOfTheWeek.MONDAY, 9.5,10.5),
    (1, DayOfTheWeek.MONDAY, 11,11.5),
    (1, DayOfTheWeek.FRIDAY, 7.5,9.5),
])


patients = [patient1, patient2, patient3, patient4, patient5, patient6, patient7, patient8, patient9, patient10]

In [ ]:
combinations = Combination(employees, patients)
c = combinations.get_combinations()

In [84]:
# Desicion variables
x = LpVariable.dicts("x", c , lowBound=0, upBound=1, cat=LpInteger)
print(x)

{(<domain.entities.Employee object at 0x0000024EEAA48C10>, <domain.entities.Patient object at 0x0000024EEAA2CB90>): x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CB90_), (<domain.entities.Employee object at 0x0000024EEAA48C10>, <domain.entities.Patient object at 0x0000024EEAA2CD50>): x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CD50_), (<domain.entities.Employee object at 0x0000024EEAA48C10>, <domain.entities.Patient object at 0x0000024EEAA2CFD0>): x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CFD0_), (<domain.entities.Employee object at 0x0000024EEAA48C10>, <domain.entities.Patient object at 0x0000024EEAA2C9D0>): x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2C9D0_), (<domain.entities.Employee object at 0x0000024EEAA48C10>, <domain.entit

In [85]:
# Maximization problem 
prob = LpProblem("Therapist Patient Assignment", LpMaximize)
print(prob)

Therapist_Patient_Assignment:
MAXIMIZE
None
VARIABLES



c:\Users\Tibo.Vanbrabandt\AppData\Local\Programs\Python\Python311\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [86]:
# Objective function to maximize te number of appointments
prob += lpSum([x[c]])

In [87]:
# Constraint to check if the .get_numeric_template() of the therapist and patient are the same
for therapist in employees:
    for patient in cp_check:
        if type(patient) == tuple:
            if therapist.schedule.get_numeric_template() == patient[0].schedule.get_numeric_template():
                prob += lpSum([x[(therapist, patient)] for patient in cp_check]) == 1
        else:
            if therapist.schedule.get_numeric_template() == patient.schedule.get_numeric_template():
                prob += lpSum([x[(therapist, patient)] for patient in cp_check]) == 1

In [88]:
# solve the problem
prob.solve()

1

In [89]:
# Print status of the problem
print("Status:", LpStatus[prob.status])

Status: Optimal


In [90]:
# Print appoinments for patienst
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2C9D0_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CB10_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CB90_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CD50_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2CFD0_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2E850_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2F350_) = 1.0
x_(<domain.entities.Employee_object_at_0x0000024EEAA48C10_,_<domain.entities.Patient_object_at_0x0000024EEAA2F5D0_) = 1.0
x_(<domain.entities.Empl